In [1]:
# coding: utf-8
# %automatically find the limits of latitude & longitute

In [2]:
import pygrib
import pandas as pd
import numpy as np
import os, time, datetime
from datetime import datetime
import glob

In [3]:
def get_distances(lats1,lons1,lats2,lons2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    earth_radius = 6371 #3958.75 miles
    #lats1, lats2 =locs_1[:,0], locs_2[:,0]
    #lons1, lons2 =locs_1[:,0], locs_2[:,0]
    #print(lats1)
    #print(lats2)
    lat_dif = np.radians(lats1 - lats2)
    long_dif = np.radians(lons1 - lons2)

    sin_d_lat = np.sin(lat_dif / 2.)
    sin_d_long = np.sin(long_dif / 2.)
    # haversine
    step_1 = (sin_d_lat ** 2) + (sin_d_long ** 2) * np.cos(np.radians(lats1)) * np.cos(np.radians(lats2))
    step_2 = 2 * np.arctan2(np.sqrt(step_1), np.sqrt(1-step_1))
    dist = step_2 * earth_radius

    return dist


In [8]:
def Tb_filtering(lat_pic,lon_pic,t_base,dist_km,Tlcl,HGT,Lndsfc_flag,thresh_neb):
    '''
    This function recieves 3 vectors of lat lon tbase and its output is Tbase
    with filtered Tbase that are too different from their environment.
    more adjustments : no. of nebs is constraining the number of neighbouring
    cells in order to make the calculation.
    '''
    filter_nebs = np.ones(t_base.shape[0])#[]
    filter_iso=filter_nebs
    lowest_dists=[]
    for i in range(0,len(lat_pic)):
        arclen= get_distances(lat_pic[i],lon_pic[i],lat_pic[:],lon_pic[:])#calc its dist from neighbours
        lowest_dists = sorted(arclen) #sort the distance from other points (degrees)
        temp_dist=np.array(lowest_dists)<dist_km
        #temp100=np.extract(np.array(lowest_dists)<100,lowest_dists)
        HGT_75th=np.percentile(temp_dist,75)
        hgt_sd = np.nanstd(temp_dist)
        temp_tbase=np.extract(temp_dist,t_base)
        Tbase_median=np.nanmedian(temp_tbase)
        #print(Tbase_median)
        no_nebs = len(temp_dist & np.isnan(t_base)==0)#no of nebs
        if no_nebs>thresh_neb: #comparison of TBase for excluding points
            if (t_base[i]-Tbase_median<-3) & (HGT[i]-HGT_75th<hgt_sd) & (Lndsfc_flag[i]==1): #there are differences that are not related to topography
                filter_nebs[i] = np.NaN
            elif (t_base[i]-Tbase_median<-3) & (Lndsfc_flag[i]== 0):
                filter_nebs[i] = np.NaN
        elif abs(Tlcl[i]-t_base[i])>5 & (no_nebs<=thresh_neb): #the point is isolated and has a large difference with LCL
            filter_iso[i] = np.NaN
        Tb_filtering = t_base*filter_nebs*filter_iso
        return Tb_filtering


In [9]:
for f in glob.glob("AUTO_5MINS_NPP_MicroPhys_20150801_0442_Table.txt"):
    print(f)
    ccn_data = pd.read_csv(f, sep='\s+',skiprows=0,header=0,na_values=[-99,-9999])
    lat_pic = ccn_data["lat"]#;%latitude col = 4
    lon_pic = ccn_data["lon"]#;%longitude col = 5
    #loc_pic = np.array([[lat_pic,lon_pic]])
    #loc_pic = np.array([[34, -81]])
    #lat_min = lat_pic.min()
    #lat_max = lat_pic.max()
    #lon_min = lon_pic.min()
    #lon_max = lat_pic.max()
    ccn_data = pd.read_csv(f, sep='\s+',skiprows=0,header=0,na_values=[-99,-9999])
    lat_pic = ccn_data["lat"]#;%latitude col = 4
    lon_pic = ccn_data["lon"]#;%longitude col = 5
    t_base = ccn_data["Tcb1.0"]#;%longitude col = 34
    Tlcl = ccn_data["T_LCL_nc"]#;%longitude col = 30
    HGT = ccn_data["HGT"]#;%longitude col = 21
    Lndsfc_flag = ccn_data["L/S"]#;%longitude col = 22
    
    Tb_f=Tb_filtering(lat_pic,lon_pic,t_base,110,Tlcl,HGT,Lndsfc_flag,10)#calc its dist from neighbours
    print(Tb_f)

        

AUTO_5MINS_NPP_MicroPhys_20150801_0442_Table.txt
0       21.0
1       18.0
2       21.0
3       20.0
4       20.0
5       24.0
6       18.0
7       23.0
8       22.0
9       23.0
10      23.0
11       NaN
12      24.0
13      23.0
14      24.0
15      22.0
16      23.0
17      24.0
18     -33.0
19      18.0
20      21.0
21      24.0
22      22.0
23       NaN
24       4.0
25      -3.0
26       3.0
27       6.0
28      23.0
29      11.0
        ... 
3122     NaN
3123   -32.0
3124   -16.0
3125    -7.0
3126     5.0
3127    -7.0
3128   -13.0
3129    -7.0
3130    -3.0
3131    -3.0
3132    -4.0
3133    -9.0
3134    -9.0
3135   -15.0
3136   -12.0
3137    -6.0
3138   -12.0
3139    -1.0
3140    -4.0
3141    -4.0
3142     0.0
3143    -8.0
3144   -12.0
3145    -7.0
3146    -3.0
3147   -17.0
3148    13.0
3149     0.0
3150    -7.0
3151    -4.0
Name: Tcb1.0, Length: 3152, dtype: float64
